<a href="https://colab.research.google.com/github/RaoSunny124/AI_201/blob/main/Langchain_RAG_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qU langchain-pinecone langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 4.2 MB/s eta 0:00:00


In [2]:
from IPython.display import Markdown, display
from google.colab import userdata
from pinecone import Pinecone, ServerlessSpec
pinecone_api_key = userdata.get('Pinecone_Api_Key')
pc = Pinecone(api_key=pinecone_api_key)


In [4]:
index_name = "practise-rag"
pc.create_index(
    name=index_name,
    dimension=768,
    metric='cosine',
    spec  = ServerlessSpec(cloud="aws",region="us-east-1"),
)

index = pc.Index(index_name)


In [8]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os

GEMINI_KEY = userdata.get("My_chatbot")

embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001", google_api_key = GEMINI_KEY)

In [9]:
vector = embeddings.embed_query("Thanks To Allah 7th Time run")

In [17]:
vector[:10]


[0.021536385640501976,
 -0.008258498273789883,
 -0.024666937068104744,
 -0.007488536648452282,
 0.03480295464396477,
 -0.010281002148985863,
 0.04205457493662834,
 -0.02350606396794319,
 0.025906207039952278,
 0.025753257796168327]

In [18]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [19]:
from langchain_core.documents import Document

document_1 = Document(
    page_content="Agentic AI is becoming increasingly important in industries like autonomous vehicles, robotics, and smart cities, where real-time decision-making and adaptability are crucial for success.",
    metadata={"topic": "Agentic AI", "category": "Industry Applications", "industries": ["Autonomous Vehicles", "Robotics", "Smart Cities"]}
)

document_2 = Document(
    page_content="LangChain is a framework for building applications with LLMs (Large Language Models), such as GPT. It provides tools to manage chains, agents, and memory for building more advanced AI applications.",
    metadata={"topic": "LangChain", "category": "AI Framework"}
)

document_3 = Document(
    page_content="The use of AI in healthcare is growing rapidly. Recent advancements in diagnostic AI tools are helping doctors identify diseases more accurately and faster, significantly improving patient outcomes.",
    metadata={"topic": "Healthcare AI", "category": "Medical Technology", "impact": "Positive"}
)

document_4 = Document(
    page_content="In the latest world news, a new tech company has developed an innovative AI that is able to solve real-world problems faster than previous models, pushing the boundaries of automation and efficiency.",
    metadata={"topic": "Tech News", "date": "2025-01-02", "company": "Innovative AI Company"}
)

document_5 = Document(
    page_content="Agentic AI refers to autonomous AI systems that are capable of decision-making, learning, and adaptation in real-world environments without needing constant human intervention.",
    metadata={"topic": "Agentic AI", "category": "Artificial Intelligence", "importance": "High"}
)

document_6 = Document(
    page_content="LangChain offers a wide range of tools to work with LLMs. This includes support for document search, question-answering systems, and memory management to make intelligent agents more effective in real-world tasks.",
    metadata={"topic": "LangChain", "category": "AI Tools", "use_case": "Advanced workflows"}
)

document_7 = Document(
    page_content="I have chocolate chip pancake and scrambled eggs for breakfast",
    metadata={"source": "personal", "meal": "breakfast"}
)

document_8 = Document(
    page_content="A new world record has been set for the fastest internet speed, with researchers breaking through previous limitations using advanced fiber-optic technology, promising faster and more efficient global communication.",
    metadata={"topic": "Tech News", "category": "Innovation", "record": "Fastest Internet Speed", "date": "2025-01-02"}
)
documents = [
    document_1, document_2, document_3, document_4, document_5,
    document_6, document_7, document_8, document_9, document_10
]

In [20]:
len(  documents)

10

In [21]:
from uuid import uuid4
uuid4()

UUID('cdbbe628-1d83-46ec-a5dc-74ffa49c768a')

In [22]:
uuids = [str(uuid4()) for i in range (len(documents))]
vector_store.add_documents(documents=documents, ids=uuids)

['4b96149f-ab94-4636-a808-4bb95d0b50bd',
 '6142b645-678a-46a7-b23c-53c888ac15e9',
 '67c10977-0793-4d54-8e4d-be1826bc60b7',
 'f08956e0-5745-426d-9bb4-b78b2f0b7138',
 '9c44d43c-5b2d-4436-8c04-203045d7f8e2',
 '33d145fb-9a03-48a8-a30d-68e4ce72c442',
 'a9fb95f7-fde9-4820-afd4-4bab735a26b4',
 '4156db6f-1903-44bd-bcb3-07c8be354a0c',
 '3f3dcfdf-b0e4-41d1-b4b2-cfa6c79c1087',
 'ee6d3036-3b82-4c53-b669-7fd23794b2c3']

In [23]:
vector_result = vector_store.similarity_search(
    "What is agentic AI?", k=7
)
print(vector_result[0].page_content)

Agentic AI refers to autonomous AI systems that are capable of decision-making, learning, and adaptation in real-world environments without needing constant human intervention.


In [34]:
def user_answer(question):

  vector_result = vector_store.similarity_search(question, k=5)
  from langchain_google_genai import ChatGoogleGenerativeAI
  from langchain.prompts import PromptTemplate

  llm = ChatGoogleGenerativeAI(
      api_key=GEMINI_KEY,
      model = "gemini-2.0-flash-exp",
      max_tokens= 100,
      temperature=0.7
  )

  prompt1= PromptTemplate(
      input_variables=["question"],
      template = "Using this data {vector_result} . Answer the following question: \n\n{question}"
  )
  chain1= prompt1 | llm

  final_answer = chain1.invoke({"vector_result": vector_result, "question": question})

  return final_answer

In [35]:
response = user_answer("What's the latest news?")

In [36]:
display(Markdown(response.content))

Based on the provided data, here's a summary of the latest news:

*   **Innovative AI:** A new tech company has developed an innovative AI that solves real-world problems faster than previous models. This is pushing the boundaries of automation and efficiency.
*   **Fastest Internet Speed:** A new world record has been set for the fastest internet speed using advanced fiber-optic technology.
*   **AI in Customer Service:** AI-powered chatbots have been shown to outperform human

In [37]:
response_1 = user_answer("What is Langchain and where it is used ?")

In [38]:
display(Markdown(response_1.content))

Based on the provided documents, here's an answer to your question:

**What is LangChain?**

LangChain is a framework designed to help build applications using Large Language Models (LLMs) like GPT. It provides tools for managing chains of operations, creating agents, and handling memory within these applications. Essentially, it helps developers create more advanced AI applications using LLMs.

**Where is LangChain used?**

LangChain is used in a variety of ways, including:

In [39]:
response_2 = user_answer("What i like in breakfast , Tell me in Key points")

In [40]:
display(Markdown(response_2.content))

Okay, based on the provided data, here's what we can extract about your breakfast preferences in key points:

*   **Chocolate chip pancake:** You enjoy chocolate chip pancakes.
*   **Scrambled eggs:** You also like to have scrambled eggs for breakfast.
